In [1]:
import torch 
import torch.nn as nn
import torchaudio
import numpy as np
from datasets import load_dataset, load_metric, Audio
import os
sr = 16000

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# hyper-parameters
TRAIN_ALL_WEIGHTS = True # すべてのパラメータを学習するか最後の全結合層だけ学習するか
lr = 1e-4 #学習率
num_train_epochs = 50 #学習するエポック数
per_device_train_batch_size = 16 # GPU1枚あたりのバッチサイズ(32の約数)
torch.backends.cudnn.benchmark = True # 再現性がなくなるが高速化
tgt = 'roman' # ラベルの種類

In [3]:
torch.__version__

'1.10.2+cu113'

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
train = pd.read_csv(f'./train_{tgt}.csv', index_col=0)
val = pd.read_csv(f'./val_{tgt}.csv', index_col=0)
train.head()

,filepath,char,hira,phone,roman
250,./data/wav/BASIC5000_0251.wav,演 奏 会 の 切 符 は 当 事 務 所 で 販 売 し て い ま す,え ん そ ー か い の き っ ぷ わ と ー じ む し ょ で は ん ば い し ...,e N s o o k a i n o k i ...,ensookainokippuwatoojimushodehanbaishiteimasu
251,./data/wav/BASIC5000_0252.wav,市 場 価 格 の 暴 落 で 一 文 無 し に な っ て し ま っ た,し じ ょ ー か か く の ぼ ー ら く で い ち も ん な し に な っ て ...,s h i j o o k a k a k u ...,shijookakakunoboorakudeichimonnashininatteshim...
252,./data/wav/BASIC5000_0253.wav,女 と い う も の は 何 で も お 金 に 換 算 し て 考 え る,お ん な と ゆ ー も の わ な ん で も お か ね に か ん ざ ん し て ...,o N n a t o y u u m o n ...,onnatoyuumonowanandemookanenikanzanshitekangaeru
253,./data/wav/BASIC5000_0254.wav,女 は テ ー ブ ル の ナ イ フ に 手 を 伸 ば し た,お ん な わ て ー ぶ る の な い ふ に て を の ば し た,o N n a w a t e e b u r ...,onnawateeburunonaifunitewonobashita
254,./data/wav/BASIC5000_0255.wav,女 の 子 は い い 服 を 見 せ び ら か す の が 好 き だ,お ん な の こ わ い い ふ く を み せ び ら か す の が す き だ,o N n a n o k o w a p a u ...,onnanokowaiifukuwomisebirakasunogasukida


In [6]:
val = val.reset_index()
train = train.reset_index()
len(train)

4750

In [7]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(f"./vocab_{tgt}.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=26, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [8]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=sr, padding_value=0.0, do_normalize=True, return_attention_mask=True)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [9]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: PreTrainedTokenizer(name_or_path='', vocab_size=26, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [10]:
class W2v2Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.pathes = df['filepath'].values
        self.sentences = df[tgt].values

    def __getitem__(self, idx):
        waveform, sample_rate = torchaudio.load(self.pathes[idx])
        batch = dict()
        batch["input_values"] = processor(waveform.reshape(-1), sampling_rate=sr).input_values[0]  
        with processor.as_target_processor():
            batch["labels"] = processor(self.sentences[idx]).input_ids       
        
        return batch

    def __len__(self):
        return len(self.df)

train_dataset = W2v2Dataset(train)
val_dataset = W2v2Dataset(val)

In [11]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [12]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [13]:
# -*- coding: utf-8 -*-

#
# レーベンシュタイン距離を用いて，
# 認識結果の誤り数を算出します．
#

import numpy as np
import copy

def calculate_error(hypothesis, reference):
    ''' レーベンシュタイン距離を計算し，
        置換誤り，削除誤り，挿入誤りを出力する
    hypothesis:       認識結果(トークン毎に区切ったリスト形式)
    reference:        正解(同上)
    total_error:      総誤り数
    substitute_error: 置換誤り数
    delete_error:     削除誤り数
    insert_error:     挿入誤り数
    len_ref:          正解文のトークン数
    '''
    # 認識結果および正解系列の長さを取得
    len_hyp = len(hypothesis)
    len_ref = len(reference)

    # 累積コスト行列を作成する
    # 行列の各要素には，トータルコスト，
    # 置換コスト，削除コスト，挿入コストの
    # 累積値が辞書形式で定義される．
    cost_matrix = [[{"total":0, 
                     "substitute":0,
                     "delete":0,
                     "insert":0} \
                     for j in range(len_ref+1)] \
                         for i in range(len_hyp+1)]

    # 0列目と0行目の入力
    for i in range(1, len_hyp+1):
        # 縦方向への遷移は，削除処理を意味する
        cost_matrix[i][0]["delete"] = i
        cost_matrix[i][0]["total"] = i
    for j in range(1, len_ref+1):
        # 横方向への遷移は，挿入処理を意味する
        cost_matrix[0][j]["insert"] = j
        cost_matrix[0][j]["total"] = j

    # 1列目と1行目以降の累積コストを計算していく
    for i in range(1, len_hyp+1):
        for j in range(1, len_ref+1):
            #
            # 各処理のコストを計算する
            #
            # 斜め方向の遷移時，文字が一致しない場合は，
            # 置換処理により累積コストが1増加
            substitute_cost = \
                cost_matrix[i-1][j-1]["total"] \
                + (0 if hypothesis[i-1] == reference[j-1] else 1)
            # 縦方向の遷移時は，削除処理により累積コストが1増加
            delete_cost = cost_matrix[i-1][j]["total"] + 1
            # 横方向の遷移時は，挿入処理により累積コストが1増加
            insert_cost = cost_matrix[i][j-1]["total"] + 1

            # 置換処理，削除処理，挿入処理のうち，
            # どの処理を行えば累積コストが最も小さくなるかを計算
            cost = [substitute_cost, delete_cost, insert_cost]
            min_index = np.argmin(cost)

            if min_index == 0:
                # 置換処理が累積コスト最小となる場合

                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = \
                    copy.copy(cost_matrix[i-1][j-1])
                # 文字が一致しない場合は，
                # 累積置換コストを1増加させる
                cost_matrix[i][j]["substitute"] \
                    += (0 if hypothesis[i-1] \
                        == reference[j-1] else 1)
            elif min_index == 1:
                # 削除処理が累積コスト最小となる場合
                
                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = copy.copy(cost_matrix[i-1][j])
                # 累積削除コストを1増加させる
                cost_matrix[i][j]["delete"] += 1
            else:
                # 置換処理が累積コスト最小となる場合
                
                # 遷移元の累積コスト情報をコピー
                cost_matrix[i][j] = copy.copy(cost_matrix[i][j-1])
                # 累積挿入コストを1増加させる
                cost_matrix[i][j]["insert"] += 1

            # 累積トータルコスト(置換+削除+挿入コスト)を更新
            cost_matrix[i][j]["total"] = cost[min_index]

    #
    # エラーの数を出力する
    # このとき，削除コストは挿入誤り，
    # 挿入コストは削除誤りになる点に注意．
    # (削除コストが1である
    #    = 1文字削除しないと正解文にならない 
    #    = 認識結果は1文字分余計に挿入されている
    #    = 挿入誤りが1である)
    #

    # 累積コスト行列の右下の要素が最終的なコストとなる．
    total_error = cost_matrix[len_hyp][len_ref]["total"]
    substitute_error = cost_matrix[len_hyp][len_ref]["substitute"]
    # 削除誤り = 挿入コスト
    delete_error = cost_matrix[len_hyp][len_ref]["insert"]
    # 挿入誤り = 削除コスト
    insert_error = cost_matrix[len_hyp][len_ref]["delete"]
    
    # 各誤り数と，正解文の文字数
    # (誤り率を算出する際に分母として用いる)を出力
    return (total_error, 
            substitute_error,
            delete_error,
            insert_error,
            len_ref)

In [14]:
def mtr_leven(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    for i in range(5):print(pred_str[i],'\n',  label_str[i])
        
    # 各誤りの総数(エラー率算出時の分子)
    total_err = 0
    total_sub = 0
    total_del = 0
    total_ins = 0
    # 正解文の総文字数(エラー率算出時の分母)
    total_length = 0
    for i in range(len(pred_str)):
        (error, substitute, delete, insert, ref_length) \
                = calculate_error(pred_str[i], label_str[i])

        # 総誤り数を累積する
        total_err += error
        total_sub += substitute
        total_del += delete
        total_ins += insert
        total_length += ref_length
        
    err_rate = 100.0 * total_err / total_length
    sub_rate = 100.0 * total_sub / total_length
    del_rate = 100.0 * total_del / total_length
    ins_rate = 100.0 * total_ins / total_length

    return {"er": err_rate, 'sr':sub_rate, 'dr':del_rate, 'ir':ins_rate}

In [15]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    #"facebook/wav2vec2-base", 
    #'facebook/wav2vec2-large-960h-lv60-self',
    #'facebook/wav2vec2-large-xlsr-53',
    'charsiu/zh_w2v2_tiny_fc_10ms',
    attention_dropout=0.2,
    hidden_dropout=0.2,
    feat_proj_dropout=0.2,
    mask_time_prob=0.1,
    layerdrop=0.2,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    diversity_loss_weight=100
)

model.lm_head = nn.Linear(384, len(processor.tokenizer))
model.config.vocab_size=len(processor.tokenizer)

In [16]:
model.config

Wav2Vec2Config {
  "_name_or_path": "charsiu/zh_w2v2_tiny_fc_10ms",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForFrameClassification"
  ],
  "attention_dropout": 0.2,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    1
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 100,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.2,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dro

In [17]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)

In [18]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

23323674

In [19]:
if TRAIN_ALL_WEIGHTS:
    for param in model.parameters():
        param.requires_grad = True
else:
    model.freeze_feature_extractor()

In [20]:
model.gradient_checkpointing_enable()

In [21]:
warmup_steps = 5 * len(train)//32
num_total_steps = num_train_epochs * len(train)//32

In [22]:
from transformers import TrainingArguments
training_args = TrainingArguments(
  output_dir=f"./wav2vec2_tiny_ja_{tgt}",
  group_by_length=False,
  per_device_train_batch_size=per_device_train_batch_size,
  gradient_accumulation_steps=32//per_device_train_batch_size,
  per_device_eval_batch_size=per_device_train_batch_size,
  evaluation_strategy="epoch",
  num_train_epochs=num_train_epochs,
  #fp16=True,
  save_strategy='epoch',
  fp16_full_eval=True,
  logging_steps=10,
  learning_rate=lr,
  warmup_steps=warmup_steps,
  save_total_limit=5,
  weight_decay=1e-5,
  dataloader_num_workers=os.cpu_count(),
  prediction_loss_only=False,
  lr_scheduler_type='linear',
  #eval_steps=10,
)
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=20,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=True,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=2,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_names=None,
label_smoothing_fac

In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=mtr_leven,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor.feature_extractor
)
trainer

In [24]:
import gc
gc.collect()

0

In [25]:
%%time
trainer.train()

***** Running training *****
  Num examples = 4750
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 7400


Epoch,Training Loss,Validation Loss,Er,Sr,Dr,Ir
0,4.132700,4.372558,95.274490,0.854701,94.419790,0.000000
1,3.701900,4.091608,96.507232,0.649244,95.857988,0.000000
2,3.280500,3.986387,96.466141,0.608153,95.857988,0.000000
3,3.261400,2.932758,98.841223,0.123274,98.717949,0.000000
4,2.764100,2.818458,100.000000,0.000000,100.000000,0.000000
5,2.373100,2.011886,99.983563,0.000000,99.983563,0.000000
6,0.756500,0.384135,13.001315,3.287311,8.612755,1.101249
7,0.473300,0.217987,6.319855,2.638067,2.646285,1.035503
8,0.362800,0.156759,4.832347,1.873767,2.358646,0.599934
9,0.320800,0.138439,3.944773,1.586128,1.545036,0.813609


***** Running Evaluation *****
  Num examples = 250
  Batch size = 16


nn 
 mizuwomareeshiakarakawanakutewanaranainodesu
nnn 
 mokuyoobiteesenkaidanwanannoshintenmonaimamashuuryooshimashita
nn 
 jooingiinwawatashigadeetawoyugametatokokuhatsushita
nn 
 isshuukanshitesononyuusuwahontooninatta
nn 
 ketsuatsuwakenkoonoparomeetaatoshitejuuyoodearu


Saving model checkpoint to ./wav2vec2_tiny_ja_roman/checkpoint-148
Configuration saved in ./wav2vec2_tiny_ja_roman/checkpoint-148/config.json
Model weights saved in ./wav2vec2_tiny_ja_roman/checkpoint-148/pytorch_model.bin
Feature extractor saved in ./wav2vec2_tiny_ja_roman/checkpoint-148/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 250
  Batch size = 16


...

Saving model checkpoint to ./wav2vec2_tiny_ja_roman/checkpoint-7252
Configuration saved in ./wav2vec2_tiny_ja_roman/checkpoint-7252/config.json
Model weights saved in ./wav2vec2_tiny_ja_roman/checkpoint-7252/pytorch_model.bin
Feature extractor saved in ./wav2vec2_tiny_ja_roman/checkpoint-7252/preprocessor_config.json
Deleting older checkpoint [wav2vec2_tiny_ja_roman/checkpoint-6512] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 250
  Batch size = 16


mizuwomareeshiakarakawanakutewanaranainodesu 
 mizuwomareeshiakarakawanakutewanaranainodesu
mokuyoobiteesenkaidanwanannoshintemonaimamashuuryooshimashita 
 mokuyoobiteesenkaidanwanannoshintenmonaimamashuuryooshimashita
jooinginnwawatashigadeetawoyugametatokokuhatsushita 
 jooingiinwawatashigadeetawoyugametatokokuhatsushita
ishuukanshitesononyuusuwahontooninatta 
 isshuukanshitesononyuusuwahontooninatta
ketsuatsuwakenkoonoparomeetaatoshitejuuyoodearu 
 ketsuatsuwakenkoonoparomeetaatoshitejuuyoodearu


Saving model checkpoint to ./wav2vec2_tiny_ja_roman/checkpoint-7400
Configuration saved in ./wav2vec2_tiny_ja_roman/checkpoint-7400/config.json
Model weights saved in ./wav2vec2_tiny_ja_roman/checkpoint-7400/pytorch_model.bin
Feature extractor saved in ./wav2vec2_tiny_ja_roman/checkpoint-7400/preprocessor_config.json
Deleting older checkpoint [wav2vec2_tiny_ja_roman/checkpoint-6660] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 1h 49min 1s, sys: 3min 17s, total: 1h 52min 18s
Wall time: 2h 1min 10s


TrainOutput(global_step=7400, training_loss=0.7016970042038608, metrics={'train_runtime': 7267.7397, 'train_samples_per_second': 32.679, 'train_steps_per_second': 1.018, 'total_flos': 5.463213955722643e+18, 'train_loss': 0.7016970042038608, 'epoch': 50.0})